In [ ]:
!pip install streamlit

In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import re
from datetime import datetime
import pytz
import pandas as pd
import json
from pandas import json_normalize
city_code = "130010" #東京
api_url = "https://weather.tsukumijima.net/api/forecast/city/"
st.title('お出かけ確認')
@st.cache
def get_data(city_code):
    df = pd.DataFrame()
    url = api_url + city_code
    try:
        response = requests.get(url)
        response.raise_for_status() 
    except requests.exceptions.RequestException as e:
           print("Error:{}".format(e))
    else:
        weather_json = response.json()
        now_hour = datetime.now(pytz.timezone('Asia/Tokyo')).hour
        if 0 <= now_hour and now_hour < 6:
            cor = weather_json['forecasts'][0]['chanceOfRain']['T00_06']
        elif 6 <= now_hour and now_hour < 12:
            cor = weather_json['forecasts'][0]['chanceOfRain']['T06_12']
        elif 12 <= now_hour and now_hour < 18:
            cor = weather_json['forecasts'][0]['chanceOfRain']['T12_18']
        else:
            cor = weather_json['forecasts'][0]['chanceOfRain']['T18_24']

        df = json_normalize(weather_json['forecasts']) 
        df = df.iloc[:,[0,1,2,10,11,12,13]]
    return df,cor
try: 
    df,cor = get_data(city_code)
    if int(cor.replace('%', ''))>50:
        st.write(f"""
        ### 現在降水確率 **{cor}** です。傘を持ちましょう。
        """)
    else:
        st.write(f"""
        ### 降水確率 **{cor}** です。
        """)
    st.subheader('天気予報')
    st.table(df)
except:
    st.error(
        "oops! error!"
    )

Writing app.py


In [3]:
from google.colab import files
files.view("/content")
files.view("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501